<a href="https://colab.research.google.com/github/Albategnius/Python/blob/master/Building_LLM_Agents_Using_LangChain_%26_OpenAI_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Setting Up Working Environment

In [1]:
# Install library yang diperlukan (jika belum terinstall)
!pip install openai langchain langchain-experimental python-dotenv
!pip install langchain
!pip install langchain_community
!pip install langchain_experimental
!pip install openai
!pip install docarray
!pip install tiktoken
!pip install wikipedia
## !pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=2b1d99f014b4dd828afb94c3f3fd4e06508d310e4d921f31e3a5788c70e1cad9
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
import os
import time
import openai
from openai import OpenAI
from langchain.chat_models import ChatOpenAI

# GANTI dengan API Key yang masih aktif
os.environ["OPENAI_API_KEY"] = "YOUR OPENAI_API_KEY"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Cek apakah API Key valid
if openai.api_key is None or openai.api_key.startswith("sk-") is False:
    raise ValueError("API Key tidak valid! Periksa kembali API Key Anda.")

# Inisialisasi model dengan versi hemat kuota
llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", openai_api_key=openai.api_key)

# Tambahkan delay untuk menghindari rate limit
time.sleep(3)

# Jalankan contoh prompt
prompt = "Jelaskan cara kerja Large Language Model (LLM) dalam 5 kalimat."
try:
    response = llm.predict(prompt)
    print("Jawaban Model:")
    print(response)
except openai.error.RateLimitError:
    print("Rate Limit Terlampaui. Coba lagi nanti atau periksa kuota API Anda.")

llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.9, model=llm_model, openai_api_key=openai.api_key)

tools = load_tools(["llm-math","wikipedia"], llm=llm)


agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)



2. Building Math Tutor Agent

In [ ]:
agent("If you have a rectangular garden with dimensions 12 meters by 8 meters, what is the total area of the garden in square meters?")

3. Building Wikipedia Search Agent

In [ ]:
question = "Andrew Ng is a British-American computer scientist \
and the Founder of Coursera and Deep Learning.ai and a Professor at Stanford University\
what books did he write?"
result = agent(question)

4. Building Python Programming Assistant Agent

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
employee_list = [["Smith", "John", 35],
                 ["Doe", "Jane", 28],
                 ["Black", "Michael", 42],
                 ["Brown", "Emily", 31],
                 ["White", "David", 39],
                 ["Green", "Sarah", 45],
                 ["Jones", "Christopher", 37]
                ]

In [ ]:
agent.run(f""" Sort these employees by their age in ascending order and \
then by last name in descending order, and print the output: {employee_list}""")

In [ ]:
## import langchain
## langchain.debug=True
## agent.run(f"""Sort these customers by \
## last name and then first name \
## and print the output: {employee_list}""")
## langchain.debug=False

5. Build Your Customized Agent

In [ ]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

agent = initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

try:
    result = agent("whats the date today?")
except:
    print("exception on external access")